# NBA Predictions by Combining Individual Player Statistics

> Note: This work was completed on April 14, 2020 as part of a capstone undergraduate project with Mingyan Zhao from Brigham Young University.

Predicting basketball games is a popular pastime for enthusiasts of the sport as well as an important job in sports analytics. NBA front offices want to know if a certain player will give them the unique edge they are looking for to make it to the next round in the playoffs. There are many factors to consider when predicting wins: players, teamwork, chemistry, and variations in individual performance all come into play. Many models look at teams as a whole and use this to predict on court success. In contrast, we have developed a method that is able to predict team wins by combining player stats and normalizing them based on engineered features. This post looks at data scraping, clearning, exploratory data anaylsis as well as different machine learning methods for game predicition using classifiers and regressors. In the end, we are able to achieve close to 70% accuracy on our test dataset.


## Our Objective

Much work has been done previously to predict wins. There are matchup predictors on popular sports websites like espn.com and fivethirtyeight.com and research papers have been written addressing the subject. For instance [1] highlights how OLS regression can be used to predict wins by incorporating strength of teams and other important factors including home team advantage, whether or not a team played back-to-back games, etc. 
Another example of game prediction comes from [2] where the authors use neural networks to help in their predictions. 

When we approached our model we wanted to do something a little bit different. Instead of focusing solely on team performance/stats in predicting wins, we wanted the ability to remove and replace players on a team and see how that affects team performance. This would greatly help game prediction because often teams go into a new season with a new combination of players or players get injured throughout the season.

More specifically we had the following questions: How can we swap out players to predict team performance? How do we adjust team stats so if a new player is hogging the ball (and minutes) it will diminish the effect of the other players around him? Can we simulate games by drawing from an empirical distribution of player stats and then combining them into team stats?

We believe this model will fit into current research by offering a unique perspective on game prediction. It will offer the ability to look at different combinations of players to find the optimal lineups. The model also gives the ability to see how playing time, shot attempts, and free throw attempts of players affects team performance. This could be especially useful for coaches, who have control of how long to play specific players (playing time) and how much they should encourage players to shoot the ball (shot attempts) and how much to drive the ball (which leads to free throw attempts). 

By converting player stats into team stats, we can effectively simulate games by drawing from an empirical distribution of past player performance and then combining our draws into team data. We can then use the team data to predict head to head matchups. This is the overall idea of our model.  

## Overview of the data

In order to create a model with these characteristics, we had to figure out how to turn player stats into team stats and what type of data we wanted to look at. In professional basketball, there are two types of stats: traditional and advanced. Traditional stats can be easily measured during the game like shots attempted and three-point shooting percentage. Advanced stats can be thought of as hand-engineered features made from traditional stats. For example effective field goal percentage, an advanced stat, takes into account that three-point field goals are worth more than two-point field goals and is normalized by field goals attempted. Advanced stats are useful because they are normalized and usually percentages. When predicting team wins, we decided to use advanced stats because of these favorable qualities. However, we need traditional and advanced player stats to create the advanced team stats.

Thus, our dataset has two parts, both from basketball-reference.com: a popular, valid source for team and player basketball stats. 

1. `Merged Player and Team Stats (Advanced and Traditional):` We use this table to draw from an empirical distribution. Once specific stats are drawn, we can convert them to team stats. This first part of the dataset covers the past two seasons (including playoffs). Drawing player statistics from too far in the past will not give a good representation of the player's current ability (the player is likely to improve or decline over long periods of time). 

![title](images/data.png)

2. `Merged Team Stats and Opposing Team Stats (Advanced):` This table goes back seven seasons. Since this data is is not labeled by team or player, we can use as much of it as we want. This is the data we use to train our logistic regression model. 

![title](images/team_data.png)

# Data Collection and Cleaning

## Data Scraping

We first got the links of all the websites that have player and games stats. Then we run through all the websites to get the player stats and team stats. We loaded data into a pandas dataframe. We then combined each player's stats with their team and opposing team's data and saved the resulting file. We also saved team data as an another file.

Here is some simplified code:

```python
# Get all the urls of all the games in last five years
month0 = ['october','november','december','january', 'february','march','april','may','june']
month1 = ['december','january', 'february','march','april','may','june']
years = [str(i) for i in range(2010,2018)][::-1]

# Set the urls
urls = ['https://www.basketball-reference.com/leagues/NBA_2018_games-'+k+'.html' for k in month]
urls = []

# Loop through all the years
for j in years:
    # 2012 had less games, so it is an exception
    if j == '2012':
        month = month1
    else:
        month = month0
    for k in month:
        urls.append('https://www.basketball-reference.com/leagues/NBA_'+j+'_games-'+k+'.html')
        
# Save URLs for all games in each month
websites = []
browser = webdriver.Chrome()
for url in urls:
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, "html.parser")
    query = soup.find_all(string='Box Score')
    tags  = ['https://www.basketball-reference.com'+ a.parent.attrs['href'] for a in query]
    websites += tags
```

Here is the code for scraping team data from the urls.

```python
def game_stats(date):

    soup1 = BeautifulSoup(browser.page_source, "html.parser")
    # Team info
    infos = soup1.find(class_='suppress_all sortable stats_table now_sortable').tbody.find_all('tr')
    
    # Get the different tables
    tables = soup1.find_all(class_='sortable stats_table now_sortable')
    
    # Get the team names and scores
    team_name1 = infos[1].find_all('td')[0].text
    score1 = infos[1].find_all('td')[-1].text
    team_name2 = infos[2].find_all('td')[0].text
    score2 = infos[2].find_all('td')[-1].text
    
    # Check and set the win condition and convert to binary output
    if int(score1) > int(score2):
        win1 = 1
        win2 = 0
    else:
        win1 = 0
        win2 = 1
        
    # Save team stats
    team_basic = []
    for k in tables[0].tfoot.find_all('td')[:-1]:
        team_basic.append(k.text)
    team_adv = []
    for k in tables[1].tfoot.find_all('td')[1:]:
        team_adv.append(k.text)
    op_basic = []
    for k in tables[2].tfoot.find_all('td')[:-1]:
        op_basic.append(k.text)
    op_adv = []
    for k in tables[3].tfoot.find_all('td')[1:]:   
        op_adv.append(k.text)
    t = np.array([team_name1,team_name2,date,win1] + team_basic+ team_adv)
    o = np.array([team_name2,team_name1,date,win2] + op_basic+ op_adv)
    return t, o
```

We scraped the player data in a similar way.


## Data Cleaning
Most of data has the right format when we saved them in pandas dataframe. We made sure that all the quantitative variables were floats. We changed the dates in the tables to pandas datetime index and minutes played to seconds played. We also changed most variable names so they were consistent and we could merge player and team data.

```python
# Fix player stat data
player_stats["date"] = pd.to_datetime(player_stats["date"], format="%Y%m%d")

# Convert from minutes played to seconds played (will be useful later)
player_stats['MP'] = [int(a)*60 + int(b) for a, b in player_stats['MP'].str.split(':')]

# Create consistent column names
player_stats = player_stats.rename({"win": "W/L TEAM", "date":"DATE", "name":"NAME", "team":"TEAM NAME", "op_team":"OPP TEAM NAME", "score":"TEAM SCORE", "op_score":"OPP TEAM SCORE", "MP":"SP"}, axis='columns')

# Put the dates in a correct format
team_stats["date"] = pd.to_datetime(team_stats["date"], format="%Y%m%d")

# Convert minutes played to seconds played
team_stats['MP'] = team_stats["MP"]*60/5

# Change the column names to be consistent
team_stats = team_stats.rename({"date":"DATE", "team":"TEAM NAME", "op_team":"OPP TEAM NAME", "MP":"SP", "win": "W/L"}, axis='columns')

# Merge the player and team stats into one table
all_stats = pd.merge(player_stats, team_stats, on=["TEAM NAME", "OPP TEAM NAME", "DATE", "W/L TEAM"], how='outer')

```

# Model Selection and Feature Engineering

## Model Selection

We determined that logistic regression was useful our application because we were trying to predict the chance of a team winning (which is fit on binary output). We knew we wanted to use advanced, normalized stats as input so we looked at the Akaike Information Criterion (AIC) of models with different feature combinations. 

After looking at different combinations of features, we had the following results:

![title](images/ft.png)

The features chosen for our model denoted in by (1). Notice that this combination of features does not have the lowest AIC. However it isn't too far away from the other models and we wanted to have a small number of features. The less features, the less variation when we convert player stats into team stats. We didn't feel that adding the extra features would help our model substantially. 

Here are the results of Logisitic Regression (with L1 Regularization) for our chosen features:

```python
best_features_aic = ['TS% TEAM', 'TOV% TEAM', 'DRB% TEAM', 'ORB% TEAM', 'TS% OPP TEAM', 'TOV% OPP TEAM']
X = team_stats.filter(best_features_aic, axis=1)
X = sm.add_constant(X,1)
y = team_stats["W/L TEAM"]

# Run logistic regression
results = sm.Logit(y, X.astype(float)).fit_regularized(disp=0)
```

![title](images/results_table.png)

## Feature Engineering
We made a few new features to help with converting player stats into team stats. They include:

`Percentage of Field Goals Attempted`: $FGA\%_i = \frac{FGA_i}{FGA_{team}}$

`Percentage of Free Throws Attempted`: $FTA\%_i = \frac{FTA_i}{FTA_{team}}$

`Percentage of Game Played`: $GP\%_i = \frac{SP_i}{SP_{team}}$

`Turnover Shot Ratio`: $TSr_i = \frac{TOV_i}{FGA_i}$

When we convert player stats to team stats, we made it so $FGA\%_i$, $FTA\%_i$, and $GP\%_i$ can be manually adjusted. 

We calculated the team stats from the player stats in the following way. Here an $i$ subscript refers to player $i$, a $team$ subscript refers to the team as whole, $opp$ refers to the opposing team, and $N$ is the number of players that played during the given game. Here $FGA$ refers to field goals attempted (the number of shots during game play), $FTA$ refers to free throws attempted, $SP$ refers to seconds played during the game, and $TOV$ refers to the number of turnovers committed during the game. For the rest of the stat definitions, please see the table below.

![title](images/table1.png)

# Data Visualization and Analysis

## Are certain stats correlated?

![title](images/DRTG_vs_TSP.png)

Here we graph player True Shooting Percentage ($TS\%$) against Defensive Rating for the Opposing Team ($DRtg_{opp}$). Both graphs show that TS% has a faint positively correlation with defensive rating. The first graph includes outliers and in the second graph we remove outliers (players that didn't attempt many free throws/field goals or missed all their shots).

## What do the stat distributions look like?

![title](images/tl.png)

In order to normalize player stats we need to be able to draw from an empirical distribution from their play for the last two seaons. The graph on the left the True Shooting Percentage Distribution for Tobias Harris of the Philadelphia 76ers. The graph on the right shows kernal density estimations (KDEs) for true shooting and free throw percentage for all players in the league.

## Who wins in head to head matchups?

![title](images/updated_predictions.png)

Once we pick players and sample their statistics, we can predict head to head matchups. On the right, the prediction was done by making 10,000 draws from each player's stats and combining them into team stats. We then ran the team stats through our model. We did not have 2 of the players' data (they were both rookies), so filled in one of their stats with a player with similar points per game. 

As shown on the left, we found out that we can also predict head to head matchups between players. It might be a stretch to compare players because the model was fit on team data, but the results are reasonable. 

## How much should we use certain players?

![title](images/rd_updated.png)

We ran simulations of the Utah Jazz against the Minnesota Timberwolves (the next team they were facing) to find the optimal playing time and field goals attempts for Donovan Mitchell and Rudy Gobert. On the left we see that as Donovan Mitchell plays more, the win probability of the Jazz goes down slightly. However the more Rudy Gobert plays, the better the team does. On the right, we see that we want Rudy Gobert to shoot the ball more and Donovan to shoot the ball less to help the team. This is important information for their coach.

## What are the best lineups for the Utah Jazz?

![title](images/best_jazz_lineup_updated.png)

We picked the 8 best players on the Jazz and looked at all the possible five-man lineups. Our best lineup hasn't played together on the Jazz before, but it seems like a reasonable lineup. Notice that all the best lineups include Rudy Gobert, which is consistent with our earlier results.

# Method

After doing exploratory data analysis, we divided our prediction method into three parts.

1. Obtain Player Stat Predictions: We tried three different ways
 * Directly using average data from past games
 * Regressing on average player data from past games
 * Drawing from an empirical distribution of past games
2. Convert Player Stats into Team Stats: This is detailed in the "Feature Engineering" section
3. Classify Team Stats to determine game winner: Use a classifier to predict whether or not the away team won 

## Obtain Player Stat Predictions

We decided to formulate the problem of predicting player stats as a regression problem instead of a time-series problem (the latter had little success). We ended up averaging the stats for the last 10 games for each player and applied regression techniques to predict there next game performance. Using average team, average opposing team, and average player stats as input, we created 14 different models to predict each of the individual player stats for any given game. 

Random forests are a good starting point because they do not require much hyperparameter tuning, and we wanted to find out if predicting player stats through regression was promising or not before we spent a lot of time on it. Regressing on player stats turned out to be the least effective way to obtain player stat predictions in the overall method of predicting games. As a result, we didn't feel like we needed to spend extra time testing more models. 

Another approach to obtain player stat predictions was simply to use their stat averages over the last 10 games. This approach actually turned out to be more effective the regressing on player stats with predicting games. This could be because we fed all of this data into a classifier that had been trained on average team data instead of regressed team data.

Lastly, obtained player stats by drawing them from an empirical distribution of past games. We drew N times from an empirical distribution over all previous games (since the 2017-2018 season) for each player. We found N=1000 to be a sufficient number of draws to promote consistent results. 

## Classify Team Stats to Determine Game Outcomes

Once we had the away and home team stats, we could combine them into one feature vector and predict the game result. We tested many common machine learning classifiers and trained them on the team stats portion of our data set. Their results can be seen in following figure. The hyper-parameters were tuned using a 5-fold cross validation on all of the methods except XGBoost. We used a 80-20 train/validation split to train this method.

![title](images/table2.png)

The data split is as follows: We used the team game data from 2017 to July 7, 2019 as our training set. We used data from July 7, 2019 to December 31, 2019 as our test set. Note that this was different than our final model test set, because in our final model we combined player statistics before predicting game outcomes. In the end, a linear support vector classifier did best overall, and we are not sure why this is the case. 

# Results

Once we trained our classifier and regression models, we ran our algorithm all the way through and recorded the results. First, we obtained the player stats in the three ways mentioned in the methods section. These include using the rolling averages, regressing on the rolling averages, and drawing from an empirical distribution. After converting the player stats to team stats, we used our SVM classifier predict game outcomes based on team stats. These results are shown in the following figures. 

![title](images/roc_curves2.png)

![title](images/table3.png)

In order achieve optimal results, we adjusted the classifier thresholds ($\tau$) to maximize accuracy. While this is infeasible to when predicting without labels, we noticed that the values of $\tau$ were often close to the average win/loss ratio for away teams. For our best model, this was around .44. This value is below .5 because it is more likely for the home team to win than the away team. We plan to use this win/loss ratio for any future predictions. We also calculated the Area Under the Curve (AUC), which may be a better representation of classifier performance in this scenario. 

Surprisingly, drawing 1000 draws from empirical player distributions was the most effective way to predict player performance for each game. Since the stats were random and a large amount were drawn, this probably created a scenario that was more robust at predicting games. 

While regressing on player stats did poorly overall, this could be because we trained the classifier on average stats instead of regressed stats. Our thinking was that our classifier, used to a less accurate dataset, would do better overall with regressed stats as input. However, regressed stats and average stats may be fundamentally different enough (regressed stats probably have higher variance) that this may have heavily affected classifier results.

## 2020 Playoffs Simulation

We put the current roster of each team and start the Playoff according to most recent rankings. The following figure is a visualization of our model's predictions put into a free online bracket. 

![title](images/model_prediction_playoffs.png)

## Player Contributions to Winning

Once we had a good way to predict game outcomes, we wanted to see which players were most important to their team's success. We did this by replaying the 2019-2020 season (up until December 31, 2019) for each team while removing different players from the team. If the team win percentage increased, that means that the removed player contributed negatively to game results. On the other hand, if the win percentage decreased, it meant that the player contributed positively to their team's success. Note that our algorithm did not take into account strength of schedule, as this would impact the win percentage of some teams more than others. 


![title](images/7.png)

We noticed an interesting pattern with these results. Almost all of the players that had the most positive impact are tall players that play close to the basket. Almost all the players that had the most negative impact are smaller players that handle the ball often. This could be because tall players may have a bigger effect on the game due to the fact that they can more easily make and block shots. It could also be that our algorithm heavily rewards actions that big-men excel at: rebounding and field goal percentage. On the other hand, smaller players turn the ball over often and our algorithm may weigh this heavily.

It is important to note that our algorithm is not perfect and there may be other factors not accounted for. For instance, while the smaller players above may negatively influence their teams in certain scenarios (probably because of shot inefficiency at times) they are also needed to help the other players succeed. Donovan Mitchell, for instance, appears on the negative side of our graph but is known to bail is team out of difficult situations where no one else can score the ball. His ability to attack the hoop opens up his teammates' options. Our algorithm probably doesn't take this into account when it simulates the Utah Jazz season without Donovan Mitchell. 

# Ethics

NBA game winning prediction is not only a interesting project to work on, but these predictions also attract people in the lottery industry. Both sports fans and speculators are interested in predictions on probabilities of game winning. This is something that many people bet money on, and many times it can lead to ethical problems. It is possible that referees and players try to fix game outcomes at times to win money. This will not only alter game outcomes, but the stats that we are using to analyze the different players. This is something we need to be aware of as we do future analysis and predictions. 

# Conclusion

In conclusion, there are many factors to consider besides overall team stats when trying to predict team performance. Our model combines individual player statistics to predict team performance and we achieve 69.60% accuracy on predictions for games in the 2019-2020 season. We are also able to analyze which players contribute the most and least to their team's success. We hope are findings are beneficial to sports fans, coaches, and even the players themselves as we all enjoy the game of basketball. 

# References

[1] Manner, Hans. "Modeling and forecasting the outcomes of NBA basketball games." Journal of Quantitative Analysis in Sports 12, no. 1 (2016): 31-41.

[2] Loeffelholz, Bernard, Earl Bednar, and Kenneth W. Bauer. "Predicting NBA games using neural networks." Journal of Quantitative Analysis in Sports 5, no. 1 (2009).

[3] Cheng, Ge & Zhang, Zhenyu & Kyebambe, Moses & Nasser, Kimbugwe. (2016). Predicting the Outcome of NBA Playoffs Based on the Maximum Entropy Principle. Entropy. 18. 450. 10.3390/e18120450. 